Hi there, we are group 65. <br>
To run the code, there are two things to be noticed: <br>
1. Put all the .csv files into a folder named "data", this "data" folder should be in the same directory as this notebook file
2. This code uses darts library, you may want to install it first. To install it, just uncomment the corresponding first few line of code in section 0 and run the cells <br>

Wish you have a great holiday. <br>
Cheers!

# 0. Import Packages & Libraries

In [134]:
# !pip install lightgbm  # you may want to install the light gbm library if it is not available locally
# !pip install darts  # you may want to install darts library if it is not available locally

import numpy as np
from darts import TimeSeries
from darts.models import LightGBMModel, MovingAverage
from darts.dataprocessing import Pipeline
from darts.dataprocessing.transformers import Scaler, StaticCovariatesTransformer, MissingValuesFiller, InvertibleMapper
from darts.utils.timeseries_generation import datetime_attribute_timeseries

from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from datetime import timedelta

# change the setting so that there won't be warning during the feature engineering
pd.options.mode.chained_assignment = None

# 1. Data Preprocessing
## 1.1 load data

In [135]:
oil_df = pd.read_csv('data/oil.csv')
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')
holiday_df = pd.read_csv('data/holidays_events.csv')
stores_df = pd.read_csv('data/stores.csv')
trans_df = pd.read_csv('data/transactions.csv')

## 1.2 data cleaning / formatting
### 1.2.1 oil data (past data)

In [136]:
# create timeseries for oil data
oil_ts = TimeSeries.from_dataframe(oil_df,time_col = 'date',value_cols = ['dcoilwtico'],freq = 'D')

# define a oil cleaning pipeline
oil_pip = Pipeline([MissingValuesFiller(),Scaler()])

# get cleaned oil data
oil_ts = oil_pip.fit_transform(oil_ts)

### 1.2.2 sales and store data (past and static data)

In [137]:
# join the training data and store data
train_store_df = pd.merge(train_df, stores_df, on='store_nbr')

# seperate data by families
train_store_df.astype({'store_nbr':'str','cluster':'str'})
train_ts_lst = TimeSeries.from_group_dataframe(train_store_df,time_col='date',value_cols='sales',group_cols=['store_nbr','family'],static_cols=['state','type','cluster','city'],fill_missing_dates=True,freq='D')
families_dict = {}
for ts in train_ts_lst:
    if ts.static_covariates_values()[0,1] in families_dict:
        families_dict[ts.static_covariates_values()[0,1]]['ts'].append(ts)
    else:
        families_dict[ts.static_covariates_values()[0,1]]= {'ts':[ts]}

for family in families_dict.keys():
    # define sales data cleaning pipeline
    sales_pip = Pipeline([MissingValuesFiller(), InvertibleMapper(np.log1p,np.expm1), Scaler(), StaticCovariatesTransformer(transformer_cat=OneHotEncoder())])
    families_dict[family]['ts'] = sales_pip.fit_transform(families_dict[family]['ts'])
    families_dict[family]['pipe'] = sales_pip

### 1.2.3 transaction data (past data)

In [138]:
trans_dict = {}
trans_df = trans_df.drop([0]) # delete the first row which is dirty data

trans_ts_lst = []
for str_nbr in range(1,55):
    tmp_trans_df = trans_df.loc[trans_df['store_nbr']==str_nbr]
    cur_ts = TimeSeries.from_dataframe(tmp_trans_df,time_col='date',value_cols='transactions',fill_missing_dates=True,freq='D')
    if cur_ts.start_time() != pd.to_datetime('2013-1-1'):
        missing_days = (cur_ts.start_time()-pd.to_datetime('2013-1-1')).days
        trans_ts_lst.append(TimeSeries.from_times_and_values(times=pd.date_range(start=pd.to_datetime('2013-1-1'),
                                                                                 end=cur_ts.start_time()-timedelta(days=1),freq='D'),values=np.zeros(missing_days)).append(cur_ts))
    else:
        trans_ts_lst.append(cur_ts)

# define transaction data cleaning pipeline and clean the data
trans_pip = Pipeline([MissingValuesFiller(),Scaler()])
trans_ts_lst = trans_pip.fit_transform(trans_ts_lst)

### 1.2.4 holiday data (static data)

In [139]:
for i in range(len(holiday_df['type'])):
    if holiday_df['transferred'][i] == True:
        holiday_df['type'][i] = 'non_holiday'
holiday_df['type'] = holiday_df['type'].replace(['Transfer','Additional','Bridge'],'Holiday')

store_holiday_lst = []
for i in range(1,55):
    store_holiday = pd.DataFrame()
    store_holiday['date'] = holiday_df['date']
    store_holiday['earthquake'] = holiday_df['description'].str.contains('Terremoto').astype(int)
    store_holiday['workday'] = (holiday_df['type']=='Work Day').astype(int)
    store_holiday['worldcup'] = holiday_df['description'].str.contains('futbol').astype(int)
    store_holiday['local_holiday'] = ((holiday_df['type']=='Holiday') & (holiday_df['locale']=='Local') & (holiday_df['locale_name']==stores_df['city'][i-1])).astype(int)
    store_holiday['regional_holiday'] = ((holiday_df['type']=='Holiday') & (holiday_df['locale']=='Regional') & (holiday_df['locale_name']==stores_df['state'][i-1])).astype(int)
    store_holiday['national_holiday'] = ((holiday_df['type']=='Holiday') & (holiday_df['locale']=='National')).astype(int)
    store_holiday['newyear'] = ((holiday_df['type']=='Holiday') & (holiday_df['description'].str.contains('Navidad'))).astype(int)
    store_holiday['black_friday'] = (holiday_df['description'].str.contains('Black Friday') | holiday_df['description'].str.contains('Cyber Monday')).astype(int)
    store_holiday['other_event'] = ((holiday_df['type']=='Event') & ~(store_holiday['worldcup'] | store_holiday['black_friday'] | store_holiday['earthquake'])).astype(int)

    store_holiday = store_holiday.loc[(store_holiday==1).any(axis=1)].groupby('date').agg('max').reset_index()
    holiday_store_ts = TimeSeries.from_dataframe(store_holiday,time_col='date',fill_missing_dates=True,freq='D',fillna_value=0).slice(pd.to_datetime('2013-1-1'),pd.to_datetime('2017-8-31'))
    store_holiday_lst.append(holiday_store_ts)

# define holiday data pipeline
holiday_pip = Pipeline([MissingValuesFiller()])
store_holiday_lst = holiday_pip.fit_transform(store_holiday_lst)

### 1.2.5 promotion data (past and future data)

In [140]:
promotion_df = pd.concat([train_df,test_df])
promotion_ts_lst = TimeSeries.from_group_dataframe(promotion_df,time_col='date',group_cols=['store_nbr','family'],value_cols='onpromotion',fill_missing_dates=True,freq='D')
promotion_dict = {}
for ts in promotion_ts_lst:
    if ts.static_covariates_values()[0,1] in promotion_dict:
        promotion_dict[ts.static_covariates_values()[0,1]]['ts'].append(ts)
    else:
        promotion_dict[ts.static_covariates_values()[0,1]]= {'ts':[ts]}

for family in promotion_dict.keys():
    # define promotion data cleaning pipeline
    promotion_pip = Pipeline([MissingValuesFiller(), Scaler()])
    promotion_dict[family]['ts'] = promotion_pip.fit_transform(promotion_dict[family]['ts'])
    promotion_dict[family]['pipe'] = promotion_pip

## 1.3 feature extraction

In [141]:
# define moving average windows
window_7 = MovingAverage(window=7)
window_14 = MovingAverage(window=14)
window_28 = MovingAverage(window=28)

### 1.3.1 oil feature

In [142]:
oil_ma_7 = window_7.filter(oil_ts)
oil_ma_7 = oil_ma_7.with_columns_renamed(col_names=oil_ma_7.columns, col_names_new="oil_ma_7")
oil_ma_28 = window_28.filter(oil_ts)
oil_ma_28 = oil_ma_28.with_columns_renamed(col_names=oil_ma_28.columns, col_names_new="oil_ma_28")
oil_ma = oil_ma_7.stack(oil_ma_28)

### 1.3.2 sales feature

In [144]:
sales_feature_dict = {}
for family in families_dict.keys():
    sales_feature_arr = []
    for sales_ts in families_dict[family]['ts']:
        sales_ma_7 = TimeSeries.from_series(window_7.filter(sales_ts).pd_series())
        sales_ma_7 = sales_ma_7.with_columns_renamed(col_names=sales_ma_7.columns, col_names_new="sales_ma_7")
        sales_ma_14 = TimeSeries.from_series(window_14.filter(sales_ts).pd_series())
        sales_ma_14 = sales_ma_14.with_columns_renamed(col_names=sales_ma_14.columns, col_names_new="sales_ma_14")
        sales_ma_28 = TimeSeries.from_series(window_28.filter(sales_ts).pd_series())
        sales_ma_28 = sales_ma_28.with_columns_renamed(col_names=sales_ma_28.columns, col_names_new="sales_ma_28")
        sales_feature_arr.append(sales_ma_7.stack(sales_ma_14).stack(sales_ma_28))
    sales_feature_dict[family] = sales_feature_arr

### 1.3.3 transaction feature

In [145]:
transactions_features = []
for tran_ts in trans_ts_lst:
    transaction_ma_7 = window_7.filter(tran_ts)
    transaction_ma_7 = transaction_ma_7.with_columns_renamed(col_names=transaction_ma_7.columns, col_names_new="transaction_ma_7")
    transaction_ma_14 = window_14.filter(tran_ts)
    transaction_ma_14 = transaction_ma_14.with_columns_renamed(col_names=transaction_ma_14.columns, col_names_new="transaction_ma_14")
    transaction_ma_28 = window_28.filter(tran_ts)
    transaction_ma_28 = transaction_ma_28.with_columns_renamed(col_names=transaction_ma_28.columns, col_names_new="transaction_ma_28")
    transaction_ma = tran_ts.with_columns_renamed(col_names=tran_ts.columns, col_names_new="transaction").stack(transaction_ma_7).stack(transaction_ma_14).stack(transaction_ma_28)
    transactions_features.append(transaction_ma)

### 1.3.4 promotion feature

In [148]:
promotion_feature_dict = {}
for family in promotion_dict.keys():
    promotion_feature_arr = []
    for promotion_ts in promotion_dict[family]['ts']:
        promotion_ma_7 = TimeSeries.from_series(window_7.filter(promotion_ts).pd_series())
        promotion_ma_7 = promotion_ma_7.with_columns_renamed(col_names=promotion_ma_7.columns, col_names_new="promotion_ma_7")
        promotion_ma_14 = TimeSeries.from_series(window_14.filter(promotion_ts).pd_series())
        promotion_ma_14 = promotion_ma_14.with_columns_renamed(col_names=promotion_ma_14.columns, col_names_new="promotion_ma_14")
        promotion_ma_28 = TimeSeries.from_series(window_28.filter(promotion_ts).pd_series())
        promotion_ma_28 = promotion_ma_28.with_columns_renamed(col_names=promotion_ma_28.columns, col_names_new="promotion_ma_28")
        promotion_feature_arr.append(promotion_ts.with_columns_renamed(col_names=promotion_ts.columns, col_names_new="promotion").stack(promotion_ma_7).stack(promotion_ma_14).stack(promotion_ma_28))
    promotion_dict[family] = promotion_feature_arr

### 1.3.5 time trend feature

In [149]:
linearTrend = TimeSeries.from_times_and_values(times=pd.date_range('2013-1-1','2017-8-31',freq='D'),values=np.arange(len(pd.date_range('2013-1-1','2017-8-31',freq='D'))),columns=["linear_trend"])
year = datetime_attribute_timeseries(time_index=pd.date_range('2013-1-1','2017-8-31',freq='D'), attribute="year")
month = datetime_attribute_timeseries(time_index=pd.date_range('2013-1-1','2017-8-31',freq='D'), attribute="month")
day = datetime_attribute_timeseries(time_index=pd.date_range('2013-1-1','2017-8-31',freq='D'), attribute="day")
dayofyear = datetime_attribute_timeseries(time_index=pd.date_range('2013-1-1','2017-8-31',freq='D'), attribute="dayofyear")
dayofweek = datetime_attribute_timeseries(time_index=pd.date_range('2013-1-1','2017-8-31',freq='D'), attribute="dayofweek")
weekofyear = datetime_attribute_timeseries(time_index=pd.date_range('2013-1-1','2017-8-31',freq='D'), attribute="weekofyear")
time_trend_feature = linearTrend.stack(year).stack(month).stack(day).stack(dayofyear).stack(dayofweek).stack(weekofyear)

# define time trend pipeline
time_trend_pip = Pipeline([Scaler()])
time_trend_train, time_trend_pred = time_trend_feature.split_before(pd.Timestamp('20170816'))
time_trend_train = time_trend_pip.fit_transform(time_trend_train)

### 1.3.6 combine all the features

In [162]:
store_feature = []

for store_nbr in range(1,55):
    store_feature.append(store_holiday_lst[store_nbr-1].stack(time_trend_feature).stack(oil_ma).stack(oil_ts))


feature_dict = {}
for family in promotion_dict.keys():
    feature_arr = []
    for i in range(54):
        feature_arr.append(promotion_dict[family][i].stack(store_feature[i]))
    feature_dict[family] = feature_arr

# 2. Model Training & Prediction
## 2.1 model training

In [179]:
from sklearn.metrics import mean_squared_log_error as msle, mean_squared_error as mse
from lightgbm import early_stopping

LGBM_Models = {}

for family in tqdm(families_dict.keys()):
    sales_ts_arr = families_dict[family]['ts']
    features = feature_dict[family]

    LGBM_Model = LightGBMModel(lags = 35,lags_future_covariates = (14,1),lags_past_covariates = [-16,-17,-18,-19,-20,-21,-22],output_chunk_length=1)
    LGBM_Model.fit(series=sales_ts_arr,future_covariates=features,past_covariates=transactions_features,verbose=True)
    LGBM_Models[family] = LGBM_Model

100%|██████████| 33/33 [02:35<00:00,  4.71s/it]


## 2.2 model prediction

In [180]:
pred_dic = {}
pred_df_arr = []
for family in tqdm(families_dict):
    sales_ts_arr = families_dict[family]['ts']
    features = feature_dict[family]

    pred = LGBM_Models[family].predict(n=16,series=sales_ts_arr,future_covariates=features,past_covariates=transactions_features)
    pred = families_dict[family]['pipe'].inverse_transform(pred,partial=True)
    for i in range(54):
        tmp_df = pred[i].pd_dataframe().reset_index()
        if (families_dict[family]['ts'][i].univariate_values()[-14:]==0).all():
            tmp_df['sales'] = [0] * 16
        tmp_df['store_nbr'] = [i+1] * 16
        tmp_df['family'] = [family] * 16
        pred_df_arr.append(tmp_df)
pred_df = pd.concat(pred_df_arr)

100%|██████████| 33/33 [00:08<00:00,  3.90it/s]


# 3. Making Submission

In [181]:
test_df['date'] = pd.to_datetime(test_df['date'])
pred_df = pd.merge(pred_df,test_df,on=['date','store_nbr','family',])
pred_df = pred_df.sort_values(by='id')
df_sub = pd.read_csv('data/sample_submission.csv', index_col='id')
df_sub.sales = pred_df['sales'].values
df_sub.to_csv('submission.csv', index=True)
